In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext('local')

# parallelize

In [3]:
my = [10, 20, 30, 40, 50]
nRdd = sc.parallelize(my) #리스트를 rdd로

In [4]:
nRdd.take(2) #action함수, = head(2)

[10, 20]

# foreach

In [5]:
nRdd.foreach(lambda v: print(v) ) #action함수, 개별 데이터 출력

In [6]:
newRdd = nRdd.map(lambda v: v+1 ) #transformation함수, (변형된)rdd 반환
newRdd.collect()

[11, 21, 31, 41, 51]

# textFile: 데이터 불러오기

In [7]:
rdd1 = sc.textFile('data/bb.txt')

In [8]:

rdd1.collect()

['10,20', '30,40', '50,60', '20,40']

# flatmap

In [9]:
rdd2 = rdd1.flatMap( lambda v: v.split(','))
rdd2.collect()

['10', '20', '30', '40', '50', '60', '20', '40']

In [10]:
rdd3 = rdd2.map(lambda v:int(v))
rdd3.collect()

[10, 20, 30, 40, 50, 60, 20, 40]

In [11]:
rdd4 = rdd3.distinct() # transformation function
rdd4.collect() # 중복되는 변수를 제거한다.

[10, 20, 30, 40, 50, 60]

In [12]:
myr = rdd1.flatMap( lambda v:v.split(',')).map(lambda v:int(v)).distinct()
myr.collect()

[10, 20, 30, 40, 50, 60]

In [13]:
myr.first() # action 함수 첫번째 결과를 가져온다.

10

In [14]:
myr.sum() # action 함수

210

In [15]:
myr.mean() # action 함수

35.0

In [16]:
myr.min()

10

In [17]:
myr.stdev()

17.07825127659933

In [18]:
myr.histogram()

TypeError: histogram() missing 1 required positional argument: 'buckets'

In [19]:
def fn( a, b):
    print('a= ', a)
    print('b= ', b)
    return a +b

In [20]:
# myr.reduce(lambda a, b : a + b)
# reduce는 transpormation 함수. lambda를 통해서 쉽게 처리할 수 있다. 
myr.reduce(fn)

210

In [21]:
dt = [(1,2),(3,4),(5,6),(1,7),(3,4)]
nRdd = sc.parallelize(dt)
nRdd.collect()

[(1, 2), (3, 4), (5, 6), (1, 7), (3, 4)]

In [22]:
newRDD = nRdd.reduceByKey(lambda a,b : a+b)
# 키가 동일한 것의 합 1:2+7=9, 3: 4 + 4, 5: 6

In [23]:
newRDD.collect()

[(1, 9), (3, 8), (5, 6)]

In [24]:
newRdd1 = nRdd.flatMap(lambda v:v)
newRdd1.collect()

[1, 2, 3, 4, 5, 6, 1, 7, 3, 4]

In [25]:
newRdd2 = nRdd.flatMap(lambda v: v).map(lambda v : (v, 1))
newRdd2.collect()                   # 

[(1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (1, 1),
 (7, 1),
 (3, 1),
 (4, 1)]

In [26]:
d = newRdd2.countByKey()
d
# action 힘수. 키를 기준으로 count를 갯수를 보여주는 dict를 반환 해준다.

defaultdict(int, {1: 2, 2: 1, 3: 2, 4: 2, 5: 1, 6: 1, 7: 1})

In [27]:
newRdd2.sortByKey().collect() # 특정 key를 기준으로 정렬
# transfor

[(1, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (3, 1),
 (4, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1)]

In [28]:
newRdd2.sortByKey(ascending=False).collect()

[(7, 1),
 (6, 1),
 (5, 1),
 (4, 1),
 (4, 1),
 (3, 1),
 (3, 1),
 (2, 1),
 (1, 1),
 (1, 1)]

In [29]:
txtRdd = sc.textFile('data/aa.txt')
txtRdd.collect()

['나는 자랑스런', '태극기 앞에', '조국과 태극기', '몸과 나는']

In [30]:
temp = txtRdd.flatMap(lambda v: v.split(' ')).collect()

In [31]:
temp

['나는', '자랑스런', '태극기', '앞에', '조국과', '태극기', '몸과', '나는']

In [32]:
d = txtRdd.flatMap(lambda v: v.split(' ')).map(lambda v: (v,1)).countByKey()
d

defaultdict(int, {'나는': 2, '자랑스런': 1, '태극기': 2, '앞에': 1, '조국과': 1, '몸과': 1})

In [33]:
txtRdd.flatMap(lambda v: v.split(' ')).map(lambda v: (v,1)).collect()

[('나는', 1),
 ('자랑스런', 1),
 ('태극기', 1),
 ('앞에', 1),
 ('조국과', 1),
 ('태극기', 1),
 ('몸과', 1),
 ('나는', 1)]

## 연습문제2

In [34]:
empList=['1,kim,000-000-000','2,lee,111-111-111',
       '3,park,222-222-222','4,song,333-333-333',
       '5,han,555-555-555','6,yoon,666-666-666']
deptList = ['1,sale','2,developer','3,manager',
           '4,sale1','5,developer1','6,manager1']

# join으로 두개의 서로 다른 데이터를 통합

In [35]:
empRdd = sc.parallelize(empList)
deptRdd = sc.parallelize(deptList)

In [36]:
empRdd.collect()

['1,kim,000-000-000',
 '2,lee,111-111-111',
 '3,park,222-222-222',
 '4,song,333-333-333',
 '5,han,555-555-555',
 '6,yoon,666-666-666']

In [37]:
deptRdd.collect()

['1,sale', '2,developer', '3,manager', '4,sale1', '5,developer1', '6,manager1']

In [38]:
empRdd1= empRdd.map(lambda v:v.split(','))
empRdd1.collect()

[['1', 'kim', '000-000-000'],
 ['2', 'lee', '111-111-111'],
 ['3', 'park', '222-222-222'],
 ['4', 'song', '333-333-333'],
 ['5', 'han', '555-555-555'],
 ['6', 'yoon', '666-666-666']]

In [39]:
empRdd2 = empRdd1.map( lambda v: (v[0], v[1]+' '+v[2]) )
empRdd2.collect()

[('1', 'kim 000-000-000'),
 ('2', 'lee 111-111-111'),
 ('3', 'park 222-222-222'),
 ('4', 'song 333-333-333'),
 ('5', 'han 555-555-555'),
 ('6', 'yoon 666-666-666')]

In [42]:
deptRdd1 = deptRdd.map(lambda v:v.split(',')).map(lambda v: (v[0], v[1]) )
deptRdd1.collect()

[('1', 'sale'),
 ('2', 'developer'),
 ('3', 'manager'),
 ('4', 'sale1'),
 ('5', 'developer1'),
 ('6', 'manager1')]

In [45]:
joinData = empRdd2.join(deptRdd1) # join은 첫번째 있는 것을 기준으로 합쳐준다.
joinData.collect()

[('1', ('kim 000-000-000', 'sale')),
 ('4', ('song 333-333-333', 'sale1')),
 ('2', ('lee 111-111-111', 'developer')),
 ('3', ('park 222-222-222', 'manager')),
 ('5', ('han 555-555-555', 'developer1')),
 ('6', ('yoon 666-666-666', 'manager1'))]

# 연습문제 3

In [46]:
weightRdd = sc.textFile('data/weight.csv')
weightRdd.collect()

['year,height,weight,grade,gender,gradecode,gendercode',
 '2017,152.5,47.9,elementary,man,2,1',
 '2017,153.2,46.6,elementary,woman,2,0',
 '2017,170.6,63.8,middle,man,0,1',
 '2017,160.4,54.2,middle,woman,0,0',
 '2017,173.9,72.3,high,man,1,1',
 '2017,160.9,57.7,high,woman,1,0']

In [60]:
weightRdd2 = weightRdd.map(lambda v:v.split(',')).filter(lambda v: v[0] == '2017')
weightRdd2.collect()

[['2017', '152.5', '47.9', 'elementary', 'man', '2', '1'],
 ['2017', '153.2', '46.6', 'elementary', 'woman', '2', '0'],
 ['2017', '170.6', '63.8', 'middle', 'man', '0', '1'],
 ['2017', '160.4', '54.2', 'middle', 'woman', '0', '0'],
 ['2017', '173.9', '72.3', 'high', 'man', '1', '1'],
 ['2017', '160.9', '57.7', 'high', 'woman', '1', '0']]

1.전체 키의 합과 평균을 구하시오
2.다음과 같이 변경하시오(학년별 갯수)
[('elementary', 2), ('middle', 2), ('high', 2)]
3.몸무게가 높은 순으로 정렬하시오

In [50]:
def fn( a, b):
    print('a= ', a)
    print('b= ', b)
    return a +b

In [69]:
weightRdd2.map(lambda x: float(x[1])).sum()


971.4999999999999

In [70]:
weightRdd2.map(lambda x: float(x[1])).mean()

161.91666666666666

In [77]:
weightRdd2.map(lambda x: (x[3],1)).countByKey()

defaultdict(int, {'elementary': 2, 'middle': 2, 'high': 2})

In [81]:
weightRdd2.sortBy(lambda x:x[1], ascending = False).collect()

[['2017', '173.9', '72.3', 'high', 'man', '1', '1'],
 ['2017', '170.6', '63.8', 'middle', 'man', '0', '1'],
 ['2017', '160.9', '57.7', 'high', 'woman', '1', '0'],
 ['2017', '160.4', '54.2', 'middle', 'woman', '0', '0'],
 ['2017', '153.2', '46.6', 'elementary', 'woman', '2', '0'],
 ['2017', '152.5', '47.9', 'elementary', 'man', '2', '1']]

In [56]:
weightRdd.map(lambda v:v.split(',')).sum(lambda v: v[1] if v >150)

SyntaxError: invalid syntax (<ipython-input-56-b7163c7fd95a>, line 1)